In [3]:
# Import dependencies
import pandas as pd
import pymongo
import requests
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd
import mysql.connector

In [4]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [5]:
counties = pd.read_excel('list-counties-us.xlsx')
counties.head()

,County or equivalent,State or district
0,Autauga County,Alabama
1,Baldwin County,Alabama
2,Barbour County,Alabama
3,Bibb County,Alabama
4,Blount County,Alabama


In [6]:
county_name = counties['County or equivalent'].str.replace(' ','_')
county_name.head()

state_name = counties['State or district']
state_name.head()

countyfullname = county_name + ',_' + state_name
countyfullname.head()

0    Autauga_County,_Alabama
1    Baldwin_County,_Alabama
2    Barbour_County,_Alabama
3       Bibb_County,_Alabama
4     Blount_County,_Alabama
dtype: object

In [7]:
# URL of page to be scraped  
reviews_list = []

#list of counties
counties = countyfullname [:10] 
print(counties)
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

0     Autauga_County,_Alabama
1     Baldwin_County,_Alabama
2     Barbour_County,_Alabama
3        Bibb_County,_Alabama
4      Blount_County,_Alabama
5     Bullock_County,_Alabama
6      Butler_County,_Alabama
7     Calhoun_County,_Alabama
8    Chambers_County,_Alabama
9    Cherokee_County,_Alabama
dtype: object


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Regina\.wdm\drivers\chromedriver\win32\88.0.4324.96]


In [8]:
img_list = []

for county in counties[:5]:
    
    url = f'https://en.wikipedia.org/wiki/{county}'
    print(url)
   
    

    #------splinter
    browser.visit(url)
    #time.sleep(3) # Sleep for 3 seconds
    html = browser.html
    #Create BeautifulSoup object; parse with 'html.parser'
    soup = BeautifulSoup(html, 'html.parser')
    
    trs = soup.find('tr', class_='mergedtoprow')
    
    for tr in trs:
        img = tr.img
        descr =  tr.div
        img_src = f"https:{img['src']}"
        img_descr = descr.text
        
        img_dict = {
            'county': county,
            'img_src': img_src,
            'img_descr': img_descr
        }
        
        img_list.append(img_dict)
    
    

https://en.wikipedia.org/wiki/Autauga_County,_Alabama
https://en.wikipedia.org/wiki/Baldwin_County,_Alabama
https://en.wikipedia.org/wiki/Barbour_County,_Alabama
https://en.wikipedia.org/wiki/Bibb_County,_Alabama
https://en.wikipedia.org/wiki/Blount_County,_Alabama


In [9]:
    img_df = pd.DataFrame(img_list)
img_df.head()

,county,img_src,img_descr
0,"Autauga_County,_Alabama",https://upload.wikimedia.org/wikipedia/commons...,Autauga County Courthouse in Prattville
1,"Baldwin_County,_Alabama",https://upload.wikimedia.org/wikipedia/commons...,The Baldwin County Courthouse in Bay Minette
2,"Barbour_County,_Alabama",https://upload.wikimedia.org/wikipedia/commons...,Barbour County courthouse in Clayton
3,"Bibb_County,_Alabama",https://upload.wikimedia.org/wikipedia/commons...,Bibb County Courthouse and Confederate monumen...
4,"Blount_County,_Alabama",https://upload.wikimedia.org/wikipedia/commons...,"Blount County Courthouse in Oneonta, Alabama"


# Save image Dataframe to AWS-Mysql

In [10]:
img_df.to_sql(name='ImagesData', con=engine, if_exists = 'append', index=False) 